In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


UsageError: Line magic function `%autoreload` not found.


In [2]:
import os

os.chdir("../")
os.getcwd()

'c:\\workspace\\EagSession1\\web_agent'

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

#### What are we going to do in this notebook?
* We are going to tets finnhub api to get stock prices for a company
* we are going to test newsapi to get news for any topic

In [4]:
import os
import finnhub
from typing import Dict, Any, List
import requests

In [5]:
def get_stock_symbol_alpha_vantage(company_name: str) -> List[Dict[str, str]]:
    """
    Get stock symbol(s) from company name using Alpha Vantage API.

    Args:
        company_name (str): The name of the company (e.g., 'Apple', 'Microsoft')

    Returns:
        List[Dict[str, str]]: List of matching companies with their symbols
            Each dictionary contains:
            - symbol: Stock symbol
            - name: Company name
            - type: Security type
            - region: Region/Exchange
            - currency: Trading currency

    """
    # Get API key from environment variable
    api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
    if not api_key:
        raise Exception("ALPHA_VANTAGE_API_KEY environment variable not set")

    # Alpha Vantage API endpoint for symbol search
    base_url = "https://www.alphavantage.co/query"
    params = {"function": "SYMBOL_SEARCH", "keywords": company_name, "apikey": api_key}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        data = response.json()
        matches = data.get("bestMatches", [])

        # Format the results
        formatted_matches = []
        for match in matches:
            formatted_matches.append(
                {
                    "symbol": match.get("1. symbol", ""),
                    "name": match.get("2. name", ""),
                    "type": match.get("3. type", ""),
                    "region": match.get("4. region", ""),
                    "currency": match.get("8. currency", ""),
                }
            )

        return formatted_matches
    except Exception as e:
        raise Exception(f"Error looking up symbol for {company_name}: {str(e)}")

In [26]:
def get_company_financials(company_symbol: str) -> Dict[str, Any]:
    """
    Get basic financials for a company using Finnhub API.

    Args:
        company_symbol (str): The symbol of the company (e.g., 'AAPL')

    Returns:
        Dict[str, Any]: Dictionary containing basic financial information

    Raises:
        Exception: If there's an error fetching the data or if the API key is not set
    """
    # Get API key from environment variable
    api_key = os.getenv("FINHUB_API_KEY")
    if not api_key:
        raise Exception("FINHUB_API_KEY environment variable not set")

    # Create a Finnhub client
    finnhub_client = finnhub.Client(api_key=api_key)

    try:
        # Get basic financials
        financials = finnhub_client.company_basic_financials(company_symbol, "all")
        # Convert metrics dictionary to markdown-style string
        metrics = financials["metric"]
        markdown_metrics = ", ".join([f"{k}: {v}" for k, v in metrics.items()])
        return markdown_metrics
    except Exception as e:
        raise Exception(f"Error fetching financials for {company_symbol}: {str(e)}")

In [27]:
company_name = "amazon"
company_details = get_stock_symbol_alpha_vantage(company_name)

In [28]:
company_symbol = company_details[0]["symbol"]
print(f"Company symbol: {company_symbol}")

Company symbol: AMZN


In [29]:
markdown_metrics = get_company_financials(company_symbol)
print(markdown_metrics)

10DayAverageTradingVolume: 41.13324, 13WeekPriceReturnDaily: -9.9243, 26WeekPriceReturnDaily: 7.885, 3MonthADReturnStd: 26.857294, 3MonthAverageTradingVolume: 39.1362, 52WeekHigh: 242.52, 52WeekHighDate: 2025-02-04, 52WeekLow: 151.61, 52WeekLowDate: 2024-08-05, 52WeekPriceReturnDaily: 11.9192, 5DayPriceReturnDaily: 3.17, assetTurnoverAnnual: 1.0209, assetTurnoverTTM: 1.1118, beta: 1.2894742, bookValuePerShareAnnual: 26.9961, bookValuePerShareQuarterly: 26.9961, bookValueShareGrowth5Y: 34.08, capexCagr5Y: 37.54, cashFlowPerShareAnnual: 3.1037, cashFlowPerShareQuarterly: 3.1037, cashFlowPerShareTTM: 6.54859, cashPerSharePerShareAnnual: 9.5537, cashPerSharePerShareQuarterly: 9.5537, currentDividendYieldTTM: None, currentEv/freeCashFlowAnnual: 64.5651, currentEv/freeCashFlowTTM: 64.5651, currentRatioAnnual: 1.0637, currentRatioQuarterly: 1.0637, dividendPerShareTTM: None, ebitdPerShareAnnual: 6.4236, ebitdPerShareTTM: 11.2704, ebitdaCagr5Y: 35.46, ebitdaInterimCagr5Y: 29.23, enterpriseValu

In [25]:
# financials['metric']

In [23]:
# Get API key from environment variable

from newsapi import NewsApiClient


def get_news_headlines(keyword: str, num_articles: int = 3):
    """
    Get news headlines and descriptions for a keyword search

    Args:
        keyword (str): Search term
        num_articles (int): Number of articles to return (default 3)

    Returns:
        List of tuples containing (title, description) for each article
    """
    api_key = os.getenv("NEWS_API_KEY")
    if not api_key:
        raise Exception("NEWS_API_KEY environment variable not set")

    # Initialize NewsAPI client
    newsapi = NewsApiClient(api_key=api_key)

    try:
        # Get all articles
        all_articles = newsapi.get_everything(
            q=keyword, language="en", sort_by="relevancy"
        )

        # Extract first n articles
        articles = all_articles["articles"][:num_articles]

        # Get title and description pairs
        headlines = [(article["title"], article["description"]) for article in articles]

        # Convert headlines to markdown format
        markdown_headlines = "\n".join(
            [f"**{title}**\n{description}\n\n" for title, description in headlines]
        )

        return markdown_headlines

    except Exception as e:
        raise Exception(f"Error fetching news for {keyword}: {str(e)}")

In [24]:
# Example usage
keyword = "Amazon"  # Can use company_name from above
headlines = get_news_headlines(keyword)
print(headlines)

**FTC Removes Posts Critical of Amazon, Microsoft, and AI Companies**
Business-guidance content published during the Biden administration has been removed from the Federal Trade Commission website.


**11 Best Early Amazon Spring Sale Deals (2025)**
Now’s your chance to save on headphones, tablets, and home gadgets.


**Amazon CEO says ‘beautiful’ new Alexa hardware is coming this fall**
Amazon is gearing up to launch new hardware to go along with its AI-upgraded Alexa. During an interview with Bloomberg, Amazon CEO Andy Jassy said the company has a “brand new lineup of devices that are coming in the fall that are beautiful.” On Wednesday, Am…


